In [1]:
import torch

import data
import utils
import info_recorder as ir
import data_loader as dl
import initializer as init
import trainer as tn
import tester as ts

In [2]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
Device name: NVIDIA GeForce GTX 1070


In [3]:
df, size_info, size_dict = data.get_df()

In [4]:
sourceCode_np = df.sourceCode.values
codeClass_np = df.classLabel.values

In [5]:
# Tokenize, build vocabulary, encode tokens
print("Tokenizing...\n")
tokenized_sourceCodes, ch2idx, max_len = utils.tokenize(sourceCode_np)
input_ids = utils.encode(tokenized_sourceCodes, ch2idx, max_len)

Tokenizing...



In [6]:
ir.record_ch2idx(ch2idx)

In [7]:
encoded_class2idx, class2idx, num_classes = utils.tokenize_encode_class(codeClass_np)

In [8]:
from sklearn.model_selection import train_test_split

train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    input_ids, encoded_class2idx, test_size = 0.1, random_state = 43
)

In [9]:
train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    train_inputs, train_labels, test_size = 0.1, random_state = 43
)

In [10]:
# Load data to PyTorch DataLoader
train_dataloader, val_dataloader, test_dataloader = dl.data_loader(train_inputs, val_inputs, test_inputs, train_labels, val_labels, test_labels, batch_size=50)

In [11]:
# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('dataDivided/tests')

In [12]:
# CNN-rand: Word vectors are randomly initialized.
tn.set_seed(42)
cnn_rand, optimizer = init.initilize_model(device=device,
                                           vocab_size=len(ch2idx),
                                           embed_dim=50,
                                           hidden_size=100,
                                           num_classes=len(class2idx),
                                           n_layers=3,
                                           dropout=0.2,
                                           learning_rate=0.25,
                                           optimizerName="Adadelta",
                                           modelType="RNN")
                                        
tn.train(device, cnn_rand, optimizer, train_dataloader, 'DataDiv_test1', writer, val_dataloader, epochs=20)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------
   1    |   2.870204   |  2.666757  |   19.03   |   97.75  
   2    |   2.376415   |  2.257241  |   30.38   |   98.60  
   3    |   2.113388   |  2.006213  |   37.46   |   98.81  
   4    |   1.903601   |  1.739835  |   47.33   |   98.86  
   5    |   1.675856   |  1.530482  |   54.16   |   98.88  
   6    |   1.449878   |  1.302152  |   61.30   |   98.84  
   7    |   1.238205   |  1.141194  |   66.58   |   98.87  
   8    |   1.072100   |  0.997378  |   71.11   |   98.89  
   9    |   0.952604   |  0.916550  |   73.36   |   98.86  
  10    |   0.868552   |  0.874556  |   74.30   |   98.86  
  11    |   0.801645   |  0.809942  |   76.45   |   98.88  
  12    |   0.749922   |  0.770692  |   77.54   |   98.85  
  13    |   0.703215   |  0.752406  |   78.24   |   98.87  
  14    |   0.664891   |  0.736054  |   78.78   |   98.86  
  15    |   0.631143

In [13]:
ts.test(device, cnn_rand, test_dataloader)

torch.Size([21000])
torch.Size([21000])
test loss:  0.6813345313781783
test acc:  80.4095238095238
